<a href="https://colab.research.google.com/github/irisawa/TF2.0-Alpha/blob/master/%E3%83%A2%E3%83%87%E3%83%AB%E3%81%AE%E4%BF%9D%E5%AD%98%E3%81%A8%E5%BE%A9%E5%85%83.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 前準備

In [0]:
!pip install -q h5py pyyaml 

##h5py
HDF5 フォーマットファイルを取り扱うための Python ライブラリー。 HDF は Hierarchical Data Format で、NumPy で扱っているデータをバイナリーファイルに保存できる。

## pyyaml
PythonでYAMLを扱うためにはPyYAMLというパッケージ

In [2]:
from __future__ import absolute_import, division, print_function

import os

!pip install -q tensorflow==2.0.0-alpha0
import tensorflow as tf
from tensorflow import keras

tf.__version__

    100% |████████████████████████████████| 79.9MB 292kB/s 
    100% |████████████████████████████████| 419kB 17.2MB/s 
    100% |████████████████████████████████| 3.0MB 9.4MB/s 
    100% |████████████████████████████████| 61kB 23.2MB/s 


'2.0.0-alpha0'

MNISTデータセットを使用して、重みの節約を実証するためにモデルをトレーニングする。これらのデモの実行をスピードアップするために、最初の1000の例だけを使用する。

画像データは (28, 28) の numpy 配列。ピクセル値が0から255なので255で割り、0〜１のデータとする。
`reshape`のマイナス１は-1とした次元の長さは他の次元の指定値から推測されて自動的に決定される。サイズの大きい配列の形状を変換するときに便利。

In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11493376/11490434 [==============================] - 0s 0us/step


In [4]:
print(train_labels.shape)
print(train_images.shape)

(1000,)
(1000, 784)


In [5]:
train_images[1]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

# モデルの定義

In [6]:
# Returns a short sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer='adam', 
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  
  return model


# Create a basic model instance
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


# チェックポイントの保存

トレーニング中やトレーニング終了時に自動的にチェックポイントを保存することができる。こうすることで、トレーニングしたモデルを再トレーニングしなくても使用できるようになる。またはトレーニングプロセスが中断された場合は、残った場所でトレーニングをピックアップすることもできる。

`tf.keras.callbacks.ModelCheckpoint`は、このタスクを実行するコールバック。

引数は以下。
- filepath	保存ファイル名
- monitor	監視値指定．例えば，monitor='val_loss'
- verbose	保存時に標準出力にコメントを出すか指定．{0, 1}
- save_best_only	精度がよくなった時だけ保存するかどうか指定．Falseの場合は毎epoch保存．
- mode 上限，下限どちらの側に収束した場合に収束判定を出すかの規定．{auto, min, max}	

In [7]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_weights_only=True,
                                                 verbose=1)

model = create_model()

model.fit(train_images, train_labels,  epochs = 10, 
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback])  # pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 864/1000 [========================>.....] - ETA: 0s - loss: 1.2028 - accuracy: 0.6690
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 1s 524us/sample - loss: 1.1139 - accuracy: 0.6940 - val_loss: 0.6991 - val_accuracy: 0.7840
Epoch 2/10
 864/1000 [========================>.....] - ETA: 0s - loss: 0.4109 - accuracy: 0.8819
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 295us/sample - loss: 0.4040 - accuracy: 0.8880 - val_loss: 0.5144 - val_accuracy: 0.8470
Epoch 3/10
 960/1000 [===========================>..] - ETA: 0s - loss: 0.2688 - accuracy: 0.9344
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 276us/sample - loss: 0.2775 - accuracy: 0.9320 - val_loss: 0.4686 - val_accuracy: 0.8540
Epoch 4/10
 896/1000 [=========================>....] - ETA: 0s - loss: 0.1970 - accuracy: 0.9531
Epoch


各エポックの終わりにチェックポイントファイルが作成される。

In [8]:
!ls {checkpoint_dir}

checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


### 未学習の新しいモデルを作成する。

モデルを復元するときは、元のモデルと同じアーキテクチャを持つモデルが必要となる。同じモデルアーキテクチャなので、別のインスタンスでも、重みを共有できる。


In [9]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 134us/sample - loss: 2.3614 - accuracy: 0.0800
Untrained model, accuracy:  8.00%


結果の通り８％と精度が悪いが
チェックポイントから重みをロードし、再評価すると重みを再利用でき良い結果が得られる。


In [10]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 89us/sample - loss: 0.4025 - accuracy: 0.8770
Restored model, accuracy: 87.70%


# コールバック

コールバックには、結果として得られるチェックポイントに一意の名前を付け、チェックポイントの頻度を調整するためのいくつかのオプションがある。

新しいモデルを学習させ、5エポックごとに一意の名前のチェックポイントを保存してみる。

In [11]:
# include the epoch in the file name. (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # Save weights, every 5-epochs.
    period=5)

model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))
model.fit(train_images, train_labels,
          epochs = 50, callbacks = [cp_callback],
          validation_data = (test_images,test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [12]:
!ls {checkpoint_dir}

checkpoint			  cp-0025.ckpt.index
cp-0000.ckpt.data-00000-of-00001  cp-0030.ckpt.data-00000-of-00001
cp-0000.ckpt.index		  cp-0030.ckpt.index
cp-0005.ckpt.data-00000-of-00001  cp-0035.ckpt.data-00000-of-00001
cp-0005.ckpt.index		  cp-0035.ckpt.index
cp-0010.ckpt.data-00000-of-00001  cp-0040.ckpt.data-00000-of-00001
cp-0010.ckpt.index		  cp-0040.ckpt.index
cp-0015.ckpt.data-00000-of-00001  cp-0045.ckpt.data-00000-of-00001
cp-0015.ckpt.index		  cp-0045.ckpt.index
cp-0020.ckpt.data-00000-of-00001  cp-0050.ckpt.data-00000-of-00001
cp-0020.ckpt.index		  cp-0050.ckpt.index
cp-0025.ckpt.data-00000-of-00001


`tf.train.latest_checkpoint`を利用すると、最新のチェックポイントファイルを探せる。

In [13]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

>注：デフォルトのtensorflowは、最新の5つのチェックポイントを保存する。


モデルをリセットして最新のチェックポイントを読み込む。

In [15]:
model = create_model()
model.load_weights(latest) #　最新の重みを読み込む
loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))


1000/1000 [==============================] - 0s 131us/sample - loss: 0.5025 - accuracy: 0.8720
Restored model, accuracy: 87.20%


# ファイルについて
上記のコードは、バイナリ形式の学習済み重みのみを含むチェックポイント形式のファイルのコレクションに重みを格納している。

チェックポイントには以下が含まれる。
- モデルの重みを1つ以上含む断片 
- どの重みがどのシャードに格納されているかを示すインデックスファイル

単一のマシンでモデルをトレーニングする場合は、接尾辞が1となる。

.data-00000-of-00001

# 手動による保存


Model.save_weightsメソッドを使用し手動で保存する。

In [16]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Restore the weights
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 126us/sample - loss: 0.5025 - accuracy: 0.8720
Restored model, accuracy: 87.20%


# モデルの保存

モデルとオプティマイザは、状態（重みと変数）とモデルの両方をファイルに保存することができる。これにより、モデルをエクスポートして、元のPythonコードにアクセスしなくても利用できる。

# HDF5ファイル

KerasはHDF5規格を使った保存フォーマットを提供する。
保存されたモデルは単一のバイナリBLOB。


In [28]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

# Save entire model to a HDF5 file
model.save('my_model.h5')

Epoch 1/5
1000/1000 [==============================] - 0s 271us/sample - loss: 1.1773 - accuracy: 0.6660
Epoch 2/5
1000/1000 [==============================] - 0s 174us/sample - loss: 0.4220 - accuracy: 0.8800
Epoch 3/5
1000/1000 [==============================] - 0s 176us/sample - loss: 0.2749 - accuracy: 0.9340
Epoch 4/5
1000/1000 [==============================] - 0s 186us/sample - loss: 0.2048 - accuracy: 0.9490
Epoch 5/5
1000/1000 [==============================] - 0s 188us/sample - loss: 0.1554 - accuracy: 0.9680



重みとオプティマイザを含めた同じモデルを再作成する。

In [29]:
# Recreate the exact same model, including weights and optimizer.
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [30]:
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 117us/sample - loss: 0.4154 - accuracy: 0.8650
Restored model, accuracy: 86.50%
